# Pandas 数据清洗任务

基于AWS Glue脚本的本地Pandas实现

## 功能
- 清洗客户基本信息
- 清洗客户行为资产数据
- 生成数据质量报告

## 优点
- ✓ 无需Java依赖
- ✓ 速度快
- ✓ 完全兼容Windows

## 环境配置

In [ ]:
import sys
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 修复Windows编码问题
if sys.platform == 'win32':
    import io
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

print("✓ 环境配置完成")

## 1. 导入库

In [ ]:
import pandas as pd
import numpy as np

# 设置Pandas显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print(f"✓ 库导入完成")
print(f"  Pandas版本: {pd.__version__}")
print(f"  NumPy版本: {np.__version__}")

## 2. 加载数据

In [ ]:
# 获取项目根目录
project_root = Path.cwd()
if project_root.name != "CASE-customer-group":
    project_root = project_root.parent

# 数据文件路径
customer_base_path = project_root / "customer_base.csv"
customer_behavior_path = project_root / "customer_behavior_assets.csv"

print(f"数据目录: {project_root}")
print(f"客户基本信息: {customer_base_path}")
print(f"客户行为资产: {customer_behavior_path}")
print()

# 加载数据
df_customer_base = pd.read_csv(customer_base_path)
df_customer_behavior = pd.read_csv(customer_behavior_path)

print(f"✓ 数据加载完成")
print(f"  客户基本信息: {df_customer_base.shape[0]} 行, {df_customer_base.shape[1]} 列")
print(f"  客户行为资产: {df_customer_behavior.shape[0]} 行, {df_customer_behavior.shape[1]} 列")

## 3. 清洗客户基本信息表

In [ ]:
# 3.1 查看原始数据
print("=" * 80)
print("客户基本信息 - 原始数据")
print("=" * 80)
print(f"\n行数: {df_customer_base.shape[0]}")
print(f"\n数据类型:")
print(df_customer_base.dtypes)
print(f"\n前3行:")
df_customer_base.head(3)

In [ ]:
# 3.2 创建副本进行清洗
df_customer_base_cleaned = df_customer_base.copy()

# 3.3 数据类型转换和标准化
# 字符串列清理（去掉首尾空格）
string_cols = ['customer_id', 'name', 'gender', 'occupation', 'occupation_type', 
               'lifecycle_stage', 'marriage_status', 'city_level', 'branch_name']

for col in string_cols:
    if col in df_customer_base_cleaned.columns:
        df_customer_base_cleaned[col] = df_customer_base_cleaned[col].str.strip()

# 数值列类型转换
df_customer_base_cleaned['age'] = pd.to_numeric(df_customer_base_cleaned['age'], errors='coerce').astype('Int64')
df_customer_base_cleaned['monthly_income'] = pd.to_numeric(df_customer_base_cleaned['monthly_income'], errors='coerce')

# 日期列转换
df_customer_base_cleaned['open_account_date'] = pd.to_datetime(df_customer_base_cleaned['open_account_date'], errors='coerce')

print("✓ 数据类型转换完成")

In [ ]:
# 3.4 异常值处理
# 年龄：18-100岁
df_customer_base_cleaned.loc[(df_customer_base_cleaned['age'] < 18) | (df_customer_base_cleaned['age'] > 100), 'age'] = np.nan

# 月收入：0-100万
df_customer_base_cleaned.loc[(df_customer_base_cleaned['monthly_income'] < 0) | (df_customer_base_cleaned['monthly_income'] > 1000000), 'monthly_income'] = np.nan

print("✓ 异常值处理完成")
print(f"  年龄异常值: {df_customer_base_cleaned['age'].isna().sum()}")
print(f"  收入异常值: {df_customer_base_cleaned['monthly_income'].isna().sum()}")

In [ ]:
# 3.5 性别标准化
df_customer_base_cleaned['gender_flag'] = df_customer_base_cleaned['gender'].apply(
    lambda x: 'valid' if x in ['男', '女'] else 'invalid'
)

invalid_gender = (df_customer_base_cleaned['gender_flag'] == 'invalid').sum()
print(f"✓ 性别标准化完成")
print(f"  性别异常值: {invalid_gender}")

In [ ]:
# 3.6 日期验证和提取
df_customer_base_cleaned['open_account_year'] = df_customer_base_cleaned['open_account_date'].dt.year
df_customer_base_cleaned['open_account_month'] = df_customer_base_cleaned['open_account_date'].dt.month

print(f"✓ 日期验证和提取完成")

In [ ]:
# 3.7 缺失值统计
print("\n缺失值统计:")
null_counts = df_customer_base_cleaned.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)

if len(null_counts) > 0:
    for col_name, count in null_counts.items():
        print(f"  {col_name}: {count}")
else:
    print("  无缺失值")

In [ ]:
# 3.8 去重（基于customer_id）
duplicates_before = len(df_customer_base_cleaned)

df_customer_base_cleaned = df_customer_base_cleaned.drop_duplicates(subset=['customer_id'])

duplicates_removed = duplicates_before - len(df_customer_base_cleaned)

print(f"✓ 去重完成")
print(f"  去重前: {duplicates_before}")
print(f"  去重后: {len(df_customer_base_cleaned)}")
print(f"  移除重复行: {duplicates_removed}")

## 4. 清洗客户行为资产表

In [ ]:
# 4.1 查看原始数据
print("=" * 80)
print("客户行为资产 - 原始数据")
print("=" * 80)
print(f"\n行数: {df_customer_behavior.shape[0]}")
print(f"\n前3行:")
df_customer_behavior.head(3)

In [ ]:
# 4.2 创建副本进行清洗
df_customer_behavior_cleaned = df_customer_behavior.copy()

# 4.3 字符串列清理
string_cols_behavior = ['id', 'customer_id', 'contact_result', 'stat_month']
for col in string_cols_behavior:
    if col in df_customer_behavior_cleaned.columns:
        df_customer_behavior_cleaned[col] = df_customer_behavior_cleaned[col].fillna('').str.strip()

# 4.4 数据类型转换
numeric_cols = [
    'total_assets', 'deposit_balance', 'financial_balance', 'fund_balance', 'insurance_balance',
    'credit_card_monthly_expense'
]
for col in numeric_cols:
    df_customer_behavior_cleaned[col] = pd.to_numeric(df_customer_behavior_cleaned[col], errors='coerce')

int_cols = [
    'deposit_flag', 'financial_flag', 'fund_flag', 'insurance_flag',
    'product_count', 'financial_repurchase_count', 'investment_monthly_count',
    'app_login_count', 'app_financial_view_time', 'app_product_compare_count'
]
for col in int_cols:
    df_customer_behavior_cleaned[col] = pd.to_numeric(df_customer_behavior_cleaned[col], errors='coerce').astype('Int64')

# 时间戳转换
df_customer_behavior_cleaned['last_app_login_time'] = pd.to_datetime(
    df_customer_behavior_cleaned['last_app_login_time'], errors='coerce'
)
df_customer_behavior_cleaned['last_contact_time'] = pd.to_datetime(
    df_customer_behavior_cleaned['last_contact_time'], errors='coerce'
)
df_customer_behavior_cleaned['marketing_cool_period'] = pd.to_datetime(
    df_customer_behavior_cleaned['marketing_cool_period'], errors='coerce'
)

print("✓ 数据类型转换完成")

In [ ]:
# 4.5 资产数据验证
df_customer_behavior_cleaned['total_assets_valid'] = df_customer_behavior_cleaned['total_assets'].apply(
    lambda x: 'valid' if (x >= 0 and x < 100000000) else 'invalid' if pd.notna(x) else np.nan
)

# 检查各类资产结余是否与总资产相符
df_customer_behavior_cleaned['assets_balance_check'] = (
    df_customer_behavior_cleaned[['deposit_balance', 'financial_balance', 'fund_balance', 'insurance_balance']]
    .sum(axis=1) > 0
).apply(lambda x: 'valid' if x else 'invalid')

invalid_assets = (df_customer_behavior_cleaned['total_assets_valid'] == 'invalid').sum()
invalid_balance = (df_customer_behavior_cleaned['assets_balance_check'] == 'invalid').sum()

print(f"✓ 资产数据验证完成")
print(f"  总资产异常值: {invalid_assets}")
print(f"  资产结余不匹配: {invalid_balance}")

In [ ]:
# 4.6 行为数据验证（非负数）
behavior_cols = [
    "credit_card_monthly_expense", "investment_monthly_count",
    "app_login_count", "app_financial_view_time", "app_product_compare_count"
]

for col_name in behavior_cols:
    df_customer_behavior_cleaned[col_name] = df_customer_behavior_cleaned[col_name].apply(
        lambda x: 0 if (pd.notna(x) and x < 0) else x
    )

print(f"✓ 行为数据验证完成")

In [ ]:
# 4.7 产品标志验证（必须为0或1）
flag_cols = ["deposit_flag", "financial_flag", "fund_flag", "insurance_flag"]
for col_name in flag_cols:
    df_customer_behavior_cleaned[col_name] = df_customer_behavior_cleaned[col_name].apply(
        lambda x: x if pd.notna(x) and x in [0, 1] else np.nan
    )

print(f"✓ 产品标志验证完成")

In [ ]:
# 4.8 缺失值处理统计 - contact_result
df_customer_behavior_cleaned['contact_result_flag'] = df_customer_behavior_cleaned['contact_result'].apply(
    lambda x: 'missing' if (pd.isna(x) or x == '') else 'present'
)

missing_contact = (df_customer_behavior_cleaned['contact_result_flag'] == 'missing').sum()

print(f"✓ 缺失值标记完成")
print(f"  contact_result 缺失值: {missing_contact}")

In [ ]:
# 4.9 去重（基于customer_id和stat_month，保留最新的记录）
duplicates_before = len(df_customer_behavior_cleaned)

# 按last_app_login_time排序，保留每个(customer_id, stat_month)的最新记录
df_customer_behavior_cleaned = df_customer_behavior_cleaned.sort_values(
    ['customer_id', 'stat_month', 'last_app_login_time'], 
    ascending=[True, True, False]
).drop_duplicates(subset=['customer_id', 'stat_month'], keep='first')

duplicates_removed = duplicates_before - len(df_customer_behavior_cleaned)

print(f"✓ 去重完成")
print(f"  去重前: {duplicates_before}")
print(f"  去重后: {len(df_customer_behavior_cleaned)}")
print(f"  移除重复行: {duplicates_removed}")

## 5. 数据质量检查报告

In [ ]:
# 收集清洗前后的数据
quality_report = {
    "timestamp": datetime.now().isoformat(),
    "job_name": "CustomerDataCleansing",
    "customer_base": {
        "input_rows": len(df_customer_base),
        "output_rows": len(df_customer_base_cleaned),
        "duplicate_removed": len(df_customer_base) - len(df_customer_base_cleaned),
        "age_invalid_count": df_customer_base_cleaned['age'].isna().sum(),
        "income_invalid_count": df_customer_base_cleaned['monthly_income'].isna().sum(),
        "gender_invalid_count": (df_customer_base_cleaned['gender_flag'] == 'invalid').sum()
    },
    "customer_behavior": {
        "input_rows": len(df_customer_behavior),
        "output_rows": len(df_customer_behavior_cleaned),
        "duplicate_removed": len(df_customer_behavior) - len(df_customer_behavior_cleaned),
        "contact_result_missing": (df_customer_behavior_cleaned['contact_result_flag'] == 'missing').sum(),
        "assets_invalid_count": (df_customer_behavior_cleaned['total_assets_valid'] == 'invalid').sum()
    }
}

# 打印质量报告
print("\n" + "=" * 80)
print("数据质量检查报告")
print("=" * 80)

print(f"\n执行时间: {quality_report['timestamp']}")
print(f"任务名称: {quality_report['job_name']}")

print(f"\n【客户基本信息】")
for key, value in quality_report["customer_base"].items():
    print(f"  {key}: {value}")

print(f"\n【客户行为资产】")
for key, value in quality_report["customer_behavior"].items():
    print(f"  {key}: {value}")

print("\n" + "=" * 80)

## 6. 输出清洗后的数据

In [ ]:
# 创建output目录
output_dir = project_root / "output"
output_dir.mkdir(exist_ok=True)

output_path_base = output_dir / "cleaned_customer_base.csv"
output_path_behavior = output_dir / "cleaned_customer_behavior.csv"

# 输出客户基本信息
df_customer_base_cleaned.to_csv(output_path_base, index=False, encoding='utf-8')
print(f"✓ 客户基本信息已输出")
print(f"  路径: {output_path_base}")
print(f"  行数: {len(df_customer_base_cleaned)}")

# 输出客户行为资产
df_customer_behavior_cleaned.to_csv(output_path_behavior, index=False, encoding='utf-8')
print(f"✓ 客户行为资产已输出")
print(f"  路径: {output_path_behavior}")
print(f"  行数: {len(df_customer_behavior_cleaned)}")

## 7. 清洗结果统计

In [ ]:
# 统计结果
print("\n" + "=" * 80)
print("清洗结果统计")
print("=" * 80)

base_clean_rate = (quality_report["customer_base"]["output_rows"] / 
                   quality_report["customer_base"]["input_rows"] * 100)
behavior_clean_rate = (quality_report["customer_behavior"]["output_rows"] / 
                       quality_report["customer_behavior"]["input_rows"] * 100)

print(f"\n客户基本信息:")
print(f"  输入行数: {quality_report['customer_base']['input_rows']}")
print(f"  输出行数: {quality_report['customer_base']['output_rows']}")
print(f"  保留率: {base_clean_rate:.2f}%")
print(f"  移除重复: {quality_report['customer_base']['duplicate_removed']}")

print(f"\n客户行为资产:")
print(f"  输入行数: {quality_report['customer_behavior']['input_rows']}")
print(f"  输出行数: {quality_report['customer_behavior']['output_rows']}")
print(f"  保留率: {behavior_clean_rate:.2f}%")
print(f"  移除重复: {quality_report['customer_behavior']['duplicate_removed']}")

print("\n" + "=" * 80)
print("✓ 数据清洗任务完成！")
print("=" * 80)

## 8. 清洗结果预览

In [ ]:
# 预览清洗后的客户基本信息
print("\n【清洗后的客户基本信息 - 前5行】")
print(df_customer_base_cleaned[[
    'customer_id', 'name', 'age', 'gender', 'monthly_income', 'open_account_date'
]].head())

In [ ]:
# 预览清洗后的客户行为资产
print("\n【清洗后的客户行为资产 - 前5行】")
print(df_customer_behavior_cleaned[[
    'customer_id', 'total_assets', 'deposit_balance', 'product_count', 'app_login_count', 'stat_month'
]].head())

## 9. 数据分析和可视化

In [ ]:
# 客户基本信息统计
print("\n" + "=" * 80)
print("客户基本信息 - 统计分析")
print("=" * 80)

print(f"\n年龄分布:")
print(df_customer_base_cleaned['age'].describe())

print(f"\n月收入分布:")
print(df_customer_base_cleaned['monthly_income'].describe())

print(f"\n性别分布:")
print(df_customer_base_cleaned['gender'].value_counts())

print(f"\n婚姻状态分布:")
print(df_customer_base_cleaned['marriage_status'].value_counts())

print(f"\n城市级别分布:")
print(df_customer_base_cleaned['city_level'].value_counts())

In [ ]:
# 客户行为资产统计
print("\n" + "=" * 80)
print("客户行为资产 - 统计分析")
print("=" * 80)

print(f"\n总资产分布:")
print(df_customer_behavior_cleaned['total_assets'].describe())

print(f"\n各类产品持有情况:")
print(f"  存款产品: {df_customer_behavior_cleaned['deposit_flag'].sum()} 人")
print(f"  理财产品: {df_customer_behavior_cleaned['financial_flag'].sum()} 人")
print(f"  基金产品: {df_customer_behavior_cleaned['fund_flag'].sum()} 人")
print(f"  保险产品: {df_customer_behavior_cleaned['insurance_flag'].sum()} 人")

print(f"\n联系结果分布:")
print(df_customer_behavior_cleaned[df_customer_behavior_cleaned['contact_result_flag'] == 'present']['contact_result'].value_counts())

print(f"\nApp登录频率统计:")
print(df_customer_behavior_cleaned['app_login_count'].describe())

## 10. 完成

In [ ]:
print("\n" + "=" * 80)
print("✓ Pandas 数据清洗任务完成！")
print("=" * 80)
print(f"\n输出文件:")
print(f"  1. {output_path_base}")
print(f"  2. {output_path_behavior}")
print(f"\n下一步:")
print(f"  - 查看 output 目录中的清洗结果")
print(f"  - 进行进一步的数据分析")
print(f"  - 构建机器学习模型")